In [3]:
from typing_extensions import TypedDict


In [2]:
%pip install langgraph

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp311-cp311-win_amd64.whl.metadata (3.0 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached zstandard-0.23.0-cp311-cp311-win_amd64.whl (495 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\piyus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
from langgraph.graph import StateGraph,START,END

In [5]:
class state(TypedDict):
    query: str
    Answer:str
    error:str
    urls:list[str] ##used in qtn 2

In [7]:
import requests

In [8]:
%pip install google-api-python-client


Note: you may need to restart the kernel to use updated packages.


In [9]:
from googleapiclient.discovery import build



        

In [ ]:
def google_search(state=state):
    
    API_KEY="AI"
    SEARCH_ENGINE_ID="b5e5"

    try:
        service=build("customsearch","v1",developerKey=API_KEY)
        results=service.cse().list(q=state["query"],cx=SEARCH_ENGINE_ID).execute()
        ans=[]
        if 'items' in results:
            for item in results['items']:
                 ans.append({
                  "title":item['title'],
                  "snippet":item['snippet'],
                  "link":item['link']
            })
            
        if not ans:
            return{"error":"No results obtained from google"}
        return{"Answer":ans}
    except Exception as e:
        return{"error":f"google failed to search:{str(e)}"}

        

In [11]:
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.


In [12]:
from tavily import TavilyClient

In [13]:
def tavilly(state=state):
    tavily_client=TavilyClient(api_key="tvly")
    try:
        response=tavily_client.search(query=state['query'])
        ans=[]
        for result in response:
            ans.append(
                {
                    "title": result['title'],
                    "snippet":result['content'],
                    "link":result['url']
                }
            )
        if not ans:
            return{"error":"Tavily failed"}
        return{"Answer":ans}
    except Exception as e:
        return{"error":"Tavily failed:str{e}"}   
    

In [24]:
%pip install duckduckgo-search

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 699.0 kB/s eta 0:00:05
   ------- -------------------------------- 0.8/4.0 MB 838.9 kB/s eta 0:00:04
   ------- -------------------------------- 0.8/4.0 MB 838.9 kB/s eta 0:00:04
   ---------- ----------------------------- 1.0/4.0 MB 786.4 kB/s eta 0:00:04
   ------------- -------------------------- 1.3/4.0 MB 818.6 kB/s eta 0:00:04
   ------------- -------------------------- 1.3/4.0 MB 818.6 kB/s eta 0:00:04
   --------------- ------------------------ 1.6/4.0 MB 822.8 kB/s eta 0:00:03
   -------------------- ------------------- 2.1/4.0 MB 955.2 kB/s eta 0:00:02
   -------------------------- -


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\piyus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
from duckduckgo_search import DDGS

In [15]:
def duck_search(state=state):
    ddgs=DDGS()
    
    try:
        results=ddgs.text(state['query'],max_results=5)
        ans=[]
        for result in results:
            ans.append({
                'title':result['title'],
                "snippet":result['body'],
                "link": result['href']
            })
        return{"Answer":ans}
    except Exception as e:
        return{"error":f"duck search failed:{str(e)}"}
        

In [16]:
def check_result(state=state):
    if "Answer" in state and state["Answer"]:
        return "Success"
    return "Fail"

In [17]:
workflow=StateGraph(state)

workflow.add_node("google_search",google_search)
workflow.add_node("tavilly",tavilly)
workflow.add_node("duck_search",duck_search)


In [18]:
workflow.add_edge(START,"google_search")
workflow.add_conditional_edges(
    "google_search",check_result,{"Success":END,"Failure":"tavilly"}
)
workflow.add_conditional_edges(
    "tavilly",check_result,{"Success":END,"Failure":"duck_search"}
)



In [19]:
workflow.add_conditional_edges(
    "duck_search",check_result,{"Success":END,"Failure":END}
)

In [62]:
workflow

In [63]:
chain=workflow.compile()

In [64]:
state=chain.invoke({"query":"whats the oldest IIT"})

In [65]:
print(state)

{'query': 'whats the oldest IIT', 'Answer': [{'title': 'Indian Institutes of Technology - Wikipedia', 'snippet': 'The Indian Institutes of Technology (IIT) are a network of engineering and technology institutions in India. Established in 1950, they are under the purview\xa0...', 'link': 'https://en.wikipedia.org/wiki/Indian_Institutes_of_Technology'}, {'title': 'History of Illinois Tech | Illinois Institute of Technology', 'snippet': 'Illinois Tech was created in 1940 by the merger of Armour Institute and Lewis Institute. Located on the West Side of Chicago.', 'link': 'https://www.iit.edu/about/history'}, {'title': 'History of Indian Institutes of Technology - Wikipedia', 'snippet': 'Contents · 1 Pre-independence developments · 2 Developments leading to the first IIT · 3 The next four IITs · 4 Establishment of IIT Guwahati and IIT Roorkee\xa0...', 'link': 'https://en.wikipedia.org/wiki/History_of_Indian_Institutes_of_Technology'}, {'title': 'History of Bronzeville | Illinois Institute 

QTN-2

In [1]:
##i will now conitnue the code for qtn:2, ill be using the code for qtn 1 in qtn 2

In [42]:
from typing_extensions import TypedDict
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever,create_retrieval_chain
from langchain import hub
from langchain_mistralai import ChatMistralAI
from langchain_huggingface import HuggingFaceEmbeddings


In [23]:
from langchain_openai import OpenAIEmbeddings

In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "AIzaSyAqZR2rJFmXP2gH4"

In [ ]:
import os
from typing import List, Dict, Any
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

def web_rag(state: Dict[str, Any]):
    
    if "Answer" not in state or not isinstance(state["Answer"], list) or not state["Answer"]:
        print("No valid search results found. Skipping RAG process.")
        return {"Answer": "sadly couldn't find any relevant information on the web."}
    
    urls = [result['link'] for result in state['Answer'] if 'link' in result]
    if not urls:
        print("No URLs found in the search results. Skipping RAG process.")
        return {"Answer": "the search results did not contain any valid links to scrape."}
        
    docs_list = []
    
    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()
            if docs and isinstance(docs[0].page_content, str):
                docs_list.extend(docs)
            else:
                print(f"Skipping URL due to invalid or empty content: {url}")
        except Exception as e:
            print(f"Failed to load or process URL {url}: {str(e)}")
            
    if not docs_list:
        return {"Answer": "unable to retrieve content from the web pages. The RAG process failed."}

    try:
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=500,
            chunk_overlap=0
        )
        doc_splits = text_splitter.split_documents(docs_list)
        
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vectorstore = FAISS.from_documents(
            documents=doc_splits,
            embedding=embedding_model
        )
        retriever = vectorstore.as_retriever()
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

        template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
        rag_prompt = ChatPromptTemplate.from_template(template)

        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | rag_prompt
            | llm
            | StrOutputParser()
        )
        
        response = rag_chain.invoke(state['query'])
        
        return {"Answer": response}

    except Exception as e:
        return {"Answer": f"An error occurred during the RAG process: {str(e)}"}

    
    

In [69]:
workflow=StateGraph(state)

workflow.add_node("google_search",google_search)
workflow.add_node("tavilly",tavilly)
workflow.add_node("duck_search",duck_search)
workflow.add_node("scraper_rag",web_rag)

In [70]:
workflow.add_edge(START, "google_search")
workflow.add_conditional_edges(
    "google_search", check_result, {"Success": "scraper_rag", "Failure": "tavilly"}
)
workflow.add_conditional_edges(
    "tavilly", check_result, {"Success": "scraper_rag", "Failure": "duck_search"}
)
workflow.add_conditional_edges(
    "duck_search", check_result, {"Success": "scraper_rag", "Failure": END}
)
workflow.add_edge("scraper_rag", END)

In [71]:
chain = workflow.compile()

In [72]:
chain.invoke({"query":"what are the latest advancements in AI?"})

{'query': 'what are the latest advancements in AI?',
 'Answer': "Based on the provided text, several advancements are mentioned:\n\n* **OpenAI's GPT-5:**  Is now available in Crescendo.\n* **Esaote's AI-powered cardiac ultrasound:** Uses machine learning to improve image clarity and diagnostic precision.\n* **Microsoft's MAI-Voice-1:**  Generates a minute of audio in under a second with minimal compute.\n* **Microsoft's MAI-1 Preview:** A foundational LLM available for public testing.\n* **Advancements in AutoML:** Automating machine learning processes.\n* **Rise of Multi-modal AI:** Combining text, image, and speech processing.\n* **Explainable AI (XAI):**  For transparency in decision-making.\n* **Enhanced Computer Vision capabilities.**\n* **Evolution of Neural Networks:** For more efficient learning.\n* **Digital Twins:** Gaining prominence for simulating real-world objects digitally.\n\nThe documents also refer to broader trends like AI in healthcare, AI-powered chips enabling fre

In [63]:
%pip install --upgrade --quiet langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 6.32.0 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32.0 which is incompatible.
